In [0]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Setting to show all columns
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 3000)
pd.set_option('display.max_colwidth', 1000)

In [0]:
! pip install sportmonks -q
! pip install tzlocal -q

In [0]:
#Day's fixtures
from sportmonks.soccer import SoccerApiV2
soccer = SoccerApiV2(api_token=
         'nEiZYXpxLywA4UWGBcsjCvFhejrrwtkRbrOnw5l0hOkqECMaj3eO7PJB9Pgc')

fixtures = soccer.fixtures_today(includes=('localTeam', 'visitorTeam'))

for f in fixtures:
   print(f['localTeam']['name'], 'plays at home against',
         f['visitorTeam']['name'])

In [0]:
# All leagues in membership
leagues = soccer.leagues(includes=('Spain'))

for l in leagues:
   print(l)

In [0]:
# All available seasons in membership
# By league_id
all_seasons = soccer.seasons()
all_seasons

In [0]:
# La Liga - Spain
spain_id = 32
laliga_id = 564
laliga_s0 = 13133  # 2018/2019
laliga_s1 = 8442   # 2017/2018
laliga_s2 = 853    # 2016/2017
laliga_s3 = 2063   # 2015/2016
laliga_s4 = 2061   # 2014/2015

# Premier League - England
england_id = 462
premier_id = 8
premier_s0 = 12962 # 2018/2019
premier_s1 = 6397  # 2017/2018
premier_s2 = 13    # 2016/2017
premier_s3 = 10    # 2015/2016
premier_s4 = 12    # 2014/2015

In [0]:
# season results
season_r = soccer.season_results(season_id = 10)
match_count = 0 # Number of matches
comm_count = 0  # Count of matches with commentaries
no_comm = {}    # Info of match w/NO commentaries
for result in season_r:
    if result['commentaries']== True:
        comm_count += 1
    else:
        no_comm = {result['commentaries'], result['id'], result['season_id'],
                    result['round_id'], result['localteam_id'],
                    result['visitorteam_id'], result['venue_id']}
    match_count += 1
    print(result)
    
print(season_r)

In [0]:
print(len(season_r)) # Number of matches
print(match_count)   # Number of matches
print(comm_count)    # Number of matches with commentaries
print(no_comm)       # No commentaries match info

In [0]:
# Commentaries for match
comments_1 = soccer.commentaries(fixture_id = 4193997)
comments_1

### La Liga 2017/2018 Season

#### Results for all matches

In [0]:
# Match Results List - La Liga 2017/2018 season
table_ll_s1 = soccer.season_results(season_id = 8442) # Results La Liga 17/18
lst_ll_s1 = []
for result in table_ll_s1:
    lst_result = [result['id'], result['round_id'], result['localteam_id'],
                  result['visitorteam_id'], result['scores']['localteam_score'],
                  result['scores']['visitorteam_score'], 
                  result['scores']['ft_score'], result['commentaries']]
    lst_ll_s1.append(lst_result)

In [5]:
# Season 1 (2017/18) DataFrame - La Liga
df_laliga_s1 = pd.DataFrame(lst_ll_s1, columns=['fixture_id', 'round_id',
                                       'localteam_id', 'visitorteam_id',
                                       'localteam_score',  'visitorteam_score',
                                       'fulltime_score', 'commentaries'])
df_laliga_s1.head()

,fixture_id,round_id,localteam_id,visitorteam_id,localteam_score,visitorteam_score,fulltime_score,commentaries
0,4193968,131630,3457,3477,1,0,1-0,True
1,4193969,131630,36,594,2,3,2-3,True
2,4193970,131630,13258,106,0,0,0-0,True
3,4193971,131630,231,7980,2,2,2-2,True
4,4193972,131630,676,528,1,1,1-1,True


In [0]:
df_laliga_s1.shape

(380, 8)

#### All Goals - 2017/18 La Liga

#### Unedited Goals Table - La Liga 2017/18

In [0]:
# Complete Table for All Goals for Season 1 (2017/18) - La Liga
goals_lst = [] # 'comment' string - split after '. '

# all fixtures from 2017/18 season table
fixtures_lst1 = df_laliga_s1.fixture_id

# Commentaries for each fixture
for fixture in fixtures_lst:
    commentaries = soccer.commentaries(fixture) 
    # Parsing data from commentaries for each fixture
    for dictionary in commentaries:
        if dictionary["goal"] == True: 
            goals_lst.append(dictionary)

In [0]:
df_fullgoals_lls1 = pd.DataFrame(goals_lst)

In [0]:
df_fullgoals_lls1[df_fullgoals_lls1['comment'].str.contains('Own Goal',
                                             regex=False, case=False, na=False)]

In [0]:
df_fullgoals_lls1

In [0]:
len(df_fullgoals_lls1[df_fullgoals_lls1['comment'].str.contains('Goal! ',
                                            regex=False, case=False, na=False)])

998

In [0]:
df_fullgoals_lls1 = df_goals_lls1[df_goals_lls1.comment.str.contains("Own Goal") == False]

In [0]:
df_fullgoals_lls1.shape

(1027, 7)

#### Goals Table - La Liga 2017/18

In [0]:
# All Goals for Season 1 (2017/18) - La Liga
import re

goal_player = []      # Scoring players
goal_team = []        # Scoring team
goal_score = []       # Updated score after goal
goal_against = []     # Teams scored against
goal_minute = []      # Minute of score
goal_fixture = []     # Fixture id of match's score
own_goal = []         # 1 = True, 0 = False
extra_minute = []     # Extra time after 90th minute

# all fixtures from 2017/18 season table
fixtures_lst = df_laliga_s1.fixture_id

# Commentaries for each fixture
for fixture in fixtures_lst:
    commentaries = soccer.commentaries(fixture) 
    
    # Parsing data from commentaries for each fixture
    for dictionary in commentaries:
        
        if dictionary["goal"] == True:        
            # setting 'comment' value to 'comment' variable
            comments = dictionary['comment']
            
            # All goals except 'Own Goal'
            if "Goal!" in comments:
                # 'score' finds digits in 'comment' string and makes digit list
                score = list(map(int, re.findall(r'\d+', comments)))
                goal_score.append(score)
                goal_minute.append(dictionary['minute']) # Adding scoring minute
                goal_fixture.append(fixture) # Adding fixture_id
                own_goal.append(0) # Not own goal = 0
                extra_minute.append(dictionary['extra_minute']) # after 90th min
                
                # adding scoring player to 'goal_player' 
                half_string = comments.split('.')[1]
                if '-' in half_string:
                    player_string = (half_string.split('-')[0]).strip(' ')
                    goal_player.append(player_string)
                elif '(' in half_string:
                    player_string = half_string.split('(')[0]
                    player_string = player_string.strip(' ')
                    goal_player.append(player_string)

                # adding scoring team to 'goal_team'
                if '-' in half_string:
                    team_string = half_string.split('-')[1]
                    team_string = (team_string.split('-')[0]).strip(' ')
                    goal_team.append(team_string)
                elif '(' in half_string:
                    team_string = half_string.split('(')[1]
                    team_string = (team_string.split(')')[0]).strip(' ')
                    goal_team.append(team_string)
                
                # adding team with goal against
                # will be useful when calculating 'clutch' score
                first_half_string = (comments.split('.')[0]).strip('Goal!')
                teams = re.sub("\d+", " ", first_half_string)
                team_1 = (teams.split(",")[0]).strip(' ')
                team_2 = (teams.split(",")[1]).strip(' ')
                if team_1 == team_string:
                    team_against = team_2
                else:
                    team_against = team_1
                goal_against.append(team_against)
        
        
            # 'Own Goal' ONLY            
            elif "Own Goal" in comments:
                # 'score' finds digits in 'comment' string and makes digit list
                score = list(map(int, re.findall(r'\d+', comments)))
                goal_score.append(score)
                goal_minute.append(dictionary['minute']) # Adding scoring minute
                goal_fixture.append(fixture) # Adding fixture_id
                own_goal.append(1) # Is an own goal = 1
                extra_minute.append(dictionary['extra_minute']) # after 90th min
                
                # adding scoring player to 'goal_player' 
                first_string = comments.split(',')[0]
                player_string = first_string.split('by ')[1]
                goal_player.append(player_string)

                # adding scoring team to 'goal_team'
                second_string = comments.split(', ')[1]
                team_string = second_string.split('.')[0]
                goal_team.append(team_string)

                # adding team with goal against
                # will be useful when calculating 'clutch' score
                half_string = comments.split('. ')[1] # <-- HERE!!
                teams = re.sub("\d+", " ", half_string)
                team_1 = (teams.split(",")[0]).strip(' ')
                team_2 = ((teams.split(",")[1]).split('.')[0]).strip(' ')
                if team_1 == team_string:
                    team_against = team_2
                else:
                    team_against = team_1
                goal_against.append(team_against)


In [0]:
goals_lls1 = [goal_player, own_goal, goal_minute, extra_minute, goal_fixture,
              goal_team, goal_against, goal_score]

In [0]:
df_goals_lls1 = pd.DataFrame(goals_lls1,
                             index=['player', 'own_goal', 'minute',
                                    'extra_minute', 'fixture_id', 'team', 
                                    'opponent', 'updated_score']).T

In [0]:
df_goals_lls1.head(6)

In [0]:
# Splitting 'updated_score' into 2 columns
df_goals_lls1['score_a'] = df_goals_lls1.updated_score.apply(lambda x: x[0])
df_goals_lls1['score_b'] = df_goals_lls1.updated_score.apply(lambda x: x[1])

In [0]:
df_goals_lls1['tied'] = (df_goals_lls1.score_a) == (df_goals_lls1.score_b)

In [0]:
# 'up_1' column set by rows with different 'fixture_id' from row below
# OR rows with same 'fixture_id' in row below with 'tied' == True
df_goals_lls1['up_1'] = ((df_goals_lls1.fixture_id.shift(-1)) != (df_goals_lls1.fixture_id)) | (df_goals_lls1.groupby('fixture_id')['tied'].shift(-1))

# 'up_2' column set by rows with same 'fixture_id' in row below 'tied' == False
# AND for same 'fixture_id' in row below 'up_1' == True
# AND current row's 'tied' == False
df_goals_lls1['up_2'] = ((df_goals_lls1.groupby('fixture_id')['tied'].shift(-1) == False) & (df_goals_lls1.groupby('fixture_id')['up_1'].shift(-1)) & (df_goals_lls1['tied'] == False))

In [0]:
df_goals_lls1['down_1'] = (((df_goals_lls1.score_a - df_goals_lls1.score_b).abs() == 1) & (df_goals_lls1.groupby('fixture_id')['up_2'].shift(-1)))

In [0]:
# Function 'event_grade()' - grading scoring events
def event_grade(row):
    # tie broken = 5
    if row['up_1'] == True:
        return 5
    # tied game = 3
    elif row['tied']:
        return 3
    # raising score by 2 = 1.5
    elif row['up_2'] == True:
        return 1.5
    # 1 away from tie = 1
    elif row['down_1'] == True:
        return 1
    else:
        return 0.5

In [0]:
def minute_weight(row):
    # Max weight = 0.2 (for minute 0 to 45)
    if row['minute'] <= 45:
        return (row['minute']*0.2) / 45
    # Max weight = 0.4 (for minute 46 to 70)
    elif 46 <= row['minute'] <= 65:
        return (row['minute']*0.4) / 65
    elif 66 <= row['minute'] <= 74:
        return (row['minute']*0.5) / 74
    # Max weight = 0.6 (for minute 71 to 80)
    elif 75 <= row['minute'] <= 80:
        return (row['minute']*0.7) / 80
    # Max weight = 0.9 (for minute 81 to 90)
    elif (81 <= row['minute'] <= 90) and row['extra_minute'] == None:
        return (row['minute']*0.9) / 90
    # Max weight = 1 (for extra minute 1 to 10)
    elif 1 <= row['extra_minute'] <= 10:
        return ((row['extra_minute']+90)*1) / 100

In [0]:
# Applying grades to scoring events
df_goals_lls1['score_grade'] = df_goals_lls1.apply(event_grade, axis=1)

# Increasing 'score_grade' for ties after previous tie was lost
df_goals_lls1.loc[df_goals_lls1['tied'] & (df_goals_lls1.groupby('fixture_id')['tied'].shift(-2)), 'score_grade'] = 4.5

# Applying weights according to minute of score
df_goals_lls1['time_weight'] = df_goals_lls1.apply(minute_weight, axis=1)

# Calculating player's clutch score
df_goals_lls1['player_clutch_score'] = df_goals_lls1['score_grade'] * df_goals_lls1['time_weight']

In [115]:
df_goals_lls1.head()

,player,own_goal,minute,extra_minute,fixture_id,team,opponent,updated_score,score_a,score_b,tied,up_1,up_2,down_1,score_grade,time_weight,player_clutch_score
0,José Luis Morales,0,88,None,4193968,Levante,Villarreal,"[1, 0]",1,0,False,True,False,False,5.0,0.880000,4.400000
1,Willian José,0,88,None,4193969,Real Sociedad,Celta de Vigo,"[2, 3]",2,3,False,True,False,False,5.0,0.880000,4.400000
2,Juanmi,0,80,None,4193969,Real Sociedad,Celta de Vigo,"[2, 2]",2,2,True,False,False,False,4.5,0.700000,3.150000
3,Maxi Gómez,0,50,None,4193969,Celta de Vigo,Real Sociedad,"[2, 1]",2,1,False,True,False,False,5.0,0.307692,1.538462
4,Mikel Oyarzabal,0,33,None,4193969,Real Sociedad,Celta de Vigo,"[1, 1]",1,1,True,False,False,False,3.0,0.146667,0.440000


In [0]:
# cluch_score means by player
#df_goals_lls1.groupby('player')['player_clutch_score'].agg(np.mean)

#### Players with more than 4 goals after 80th minute

In [117]:
df_80min_lls1 = df_goals_lls1[df_goals_lls1.minute > 80].groupby('player').filter(lambda x: len(x) >= 4).sort_values(['player'])
df_80min_lls1

,player,own_goal,minute,extra_minute,fixture_id,team,opponent,updated_score,score_a,score_b,tied,up_1,up_2,down_1,score_grade,time_weight,player_clutch_score
874,Cristiano Ronaldo,0,87,None,4194296,Real Madrid,Athletic Club,"[1, 1]",1,1,True,False,False,False,3.0,0.87,2.610
730,Cristiano Ronaldo,0,84,None,4194238,Real Madrid,Eibar,"[1, 2]",1,2,False,True,False,False,5.0,0.84,4.200
196,Cristiano Ronaldo,0,85,None,4194039,Real Madrid,Getafe,"[1, 2]",1,2,False,True,False,False,5.0,0.85,4.250
512,Cristiano Ronaldo,0,84,None,4194158,Real Madrid,Deportivo de La Coruña,"[6, 1]",6,1,False,False,False,False,0.5,0.84,0.420
774,Cristiano Ronaldo,0,90,1,4194256,Real Madrid,Girona,"[6, 3]",6,3,False,False,False,False,0.5,0.91,0.455
648,Daniel Parejo,0,85,None,4194202,Valencia,Málaga,"[1, 2]",1,2,False,True,False,False,5.0,0.85,4.250
833,Daniel Parejo,0,87,None,4194278,Valencia,Barcelona,"[2, 1]",2,1,False,False,False,True,1.0,0.87,0.870
615,Daniel Parejo,0,89,None,4194191,Valencia,Levante,"[3, 1]",3,1,False,False,True,False,1.5,0.89,1.335
403,Daniel Parejo,0,81,None,4194114,Valencia,Celta de Vigo,"[2, 1]",2,1,False,True,False,False,5.0,0.81,4.050
566,Lionel Messi,0,84,None,4194174,Barcelona,Alavés,"[2, 1]",2,1,False,True,False,False,5.0,0.84,4.200


In [118]:
df_80min_lls1.shape

(30, 17)

In [121]:
df_80min_lls1.groupby('player')['player_clutch_score'].agg(np.mean)

player
Cristiano Ronaldo    2.387000
Daniel Parejo        2.626250
Lionel Messi         1.954375
Luis Suárez          1.058750
Paulinho             1.623750
Ángel Rodríguez      2.131000
Name: player_clutch_score, dtype: float64

In [0]:
#df_goals_lls1[df_goals_lls1.minute.between(80, 100)]